In [1]:
# Import our dependencies
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

data = Path('Resourses/patient_data.csv')
df = pd.read_csv(data)
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,...,glyburide-metformin:Up,A1Cresult:>7,A1Cresult:>8,A1Cresult:None,A1Cresult:Norm,max_glu_serum:>200,max_glu_serum:>300,max_glu_serum:None,max_glu_serum:Norm,readmitted
0,4,66,3,18,0,0,1,9,1,1,...,0,0,0,1,0,0,0,1,0,1
1,2,48,0,15,4,0,0,7,0,0,...,0,0,0,1,0,0,0,1,0,0
2,4,21,3,23,1,0,2,7,0,1,...,0,0,0,1,0,0,0,1,0,0
3,5,38,0,5,0,0,0,2,1,1,...,0,0,1,0,0,0,0,1,0,0
4,1,6,0,6,0,0,0,6,0,1,...,0,0,0,1,0,0,0,1,0,1


In [2]:
# Split target column from dataset
y = df['readmitted']
X = df.drop(columns='readmitted')

In [3]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create scaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [4]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # match the number of input layers
    input_features = len(X_train.columns)

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Input(shape=[input_features]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [5]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from .\untitled_project\tuner0.json


In [6]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 00m 48s]
val_accuracy: 0.6308321356773376

Best val_accuracy So Far: 0.6318541169166565
Total elapsed time: 3d 18h 51m 02s


In [7]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 3,
 'num_layers': 1,
 'units_0': 5,
 'units_1': 9,
 'units_2': 9,
 'units_3': 9,
 'units_4': 9,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0047'}

In [8]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

796/796 - 1s - 771us/step - accuracy: 0.6319 - loss: 0.6428
Loss: 0.6428183317184448, Accuracy: 0.6318541169166565


In [9]:
# Train the model
fit_model = best_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 608us/step - accuracy: 0.6405 - loss: 0.6333
Epoch 2/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 602us/step - accuracy: 0.6418 - loss: 0.6291
Epoch 3/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 600us/step - accuracy: 0.6420 - loss: 0.6300
Epoch 4/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 615us/step - accuracy: 0.6400 - loss: 0.6321
Epoch 5/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 594us/step - accuracy: 0.6438 - loss: 0.6294
Epoch 6/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 608us/step - accuracy: 0.6406 - loss: 0.6305
Epoch 7/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step - accuracy: 0.6428 - loss: 0.6298
Epoch 8/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step - accuracy: 0.6454 - loss: 0.6269
Epoch 9/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 595us/step - accuracy: 0.6401 - loss: 0.6303
Epoch 10/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 0.6423 - loss: 0.6292
Epoch 11/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 1s 597us/step - accuracy: 0.6420 - loss: 0.62

In [14]:
# Evaluate the model using the test data
model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Accuracy: {model_accuracy}")

796/796 - 0s - 573us/step - accuracy: 0.6309 - loss: 0.6433
Accuracy: [0.643333375453949, 0.6308714151382446]


In [16]:
# Export our model to HDF5 file
best_model.save("NN_1.h5")